In [1]:
import numpy as np
import matplotlib.pyplot as plt
from timeit import default_timer as timer
import random
import cmath

def pdf(a,x):                           # Probability Distribution 
    y=(0.5/a) * np.exp(-abs(x)/a)
    return y

def omega_list(no_of_osc, a):           # To generate a lorentzian weighted distribution of w.
    N=0 
    x_list=[]
    while(N!=no_of_osc):
        x=random.uniform(-8,8)          # Generate a uniform random number between some apt finite range of x.
        y=random.uniform(0,1)           # Generate a uniform random number between 0 and y_max
        if(pdf(a,x)>y):                 # if the random y value < pdf(x), then accept that point.
            x_list.append(x)
            N=N+1
    x_list=np.array(x_list)
    return x_list

def rpsi(theta_list):                 # to find r(t) and psi(t) for a given time t 
    z=np.mean(np.exp([complex(0,theta) for theta in theta_list]))
    r=abs(z)
    psi=cmath.phase(z)    
    return r,psi

def evolution(theta_list,w_list,k,r,psi,dt):   # Integration function for theta_i's
    new_theta=np.add(theta_list,dt*np.add(w_list,k*r*np.sin(np.subtract(psi,theta_list))))
    return new_theta

def random_theta_i(N):
    anglelist=np.zeros(N, dtype=float)
    for i in range(N):
        theta=random.random()*2*np.pi
        anglelist[i]+=theta
    return anglelist

In [ ]:
for sigma in np.arange(1.5,5,1):
    r_list=[]
    k_list=[]
    start = timer()
    for k in np.concatenate((np.concatenate((np.arange(0,0,0.5),np.arange(0,0,0.2))),np.arange(0,10.5,0.2))):
        list_to_avg=[]
        for j in range(1):            # average over values of r for same value of k (set at 1 for now) 
            dt=10**(-2)               # delta t                 
            time_steps=4*(10**3)      # Number of Time Steps
            N=2*10**3                 # Number of Oscillators
            rlist=np.zeros(int(time_steps), dtype=float)
            w_list=omega_list(N, sigma)               # Initial Distribution of angular frequencies.
            theta_list=random_theta_i(N)   # Initial Random Distribution of Phases.
            
            for t in range(int(time_steps)):
                r,psi=rpsi(theta_list)
                theta_list=evolution(theta_list,w_list,k,r,psi,dt)      # New theta Phase list.
                rlist[t]+=r  
            r_mean=np.mean(rlist[int(np.floor(len(rlist)*3/4)):int(len(rlist))])   # Mean over the last 25% elements
            list_to_avg.append(r_mean)
        
        r_list.append(np.mean(np.array(list_to_avg)))
        k_list.append(k)
            
    end = timer()
    
    # Ugly code to find K_c
    print("Runtime= ", end-start, " secs")
    kc_list_a=np.zeros(len(k_list)-1,dtype=float)
    for i in range(len(k_list)-1):
        kc_list_a[i]=kc_list_a[i]+(r_list[i+1]-r_list[i])
    kc_index_a=np.where(kc_list_a==np.amax(kc_list_a))[0][0]
    kc_a=k_list[kc_index_a]
    
    kc_list_b=np.zeros(len(k_list),dtype=float)
    for j in range(len(k_list)):
        kc_list_b[j]=kc_list_b[j]+abs(r_list[j]-0.1)
    kc_index_b=np.where(kc_list_b==np.min(kc_list_b))[0][0]
    kc_b=k_list[kc_index_b]
    
    kc=np.round((kc_a+kc_b)/2,3)
    # Ugly code to find K_c OVER
    
    
    # Ugly code to plot and save
    fig= plt.figure()
    plt.plot(k_list,r_list,'o-', color="#118df0")
    plt.plot([kc,kc],[0,1],'--', color="#000000")
    plt.grid(True)
    plt.ylabel("R at equilibrium")
    plt.xlabel("K ")
    ax = fig.add_subplot(111)
    ax.text(0.7*k,0.3,"K_c = " + str (round(kc,4)) ,color='black', fontsize=12)
    ax.text(0.7*k,0.2,"σ (Kink)= " + str(sigma) ,color='black', fontsize=12)
    plt.savefig('plot ' + "Kink " + "σ=" + str(sigma) + " dt=" + str(dt) + " time="+ str(time_steps*dt)+'.png', dpi=(200))  
    plt.show()    
    # Ugly code to plot and save OVER